In [1]:
# Music processing
from music21 import *
# Algorithms
from itertools import takewhile
# File management
import os
from glob import glob
from pathlib import Path
import csv

## Settings

In [2]:
# Classical music target and destination
#corpus_path = 'C:\\Users\\alext\\Desktop\\School\\2021 Spring\\CS 271\\Final Project\\Feature Extraction\\Classical\\Classical Corpus'
#export_path = 'C:\\Users\\alext\\Desktop\\School\\2021 Spring\\CS 271\\Final Project\\Feature Extraction\\Classical\\Classical Extracted Notes'

# Jazz music target and destination
corpus_path = 'C:\\Users\\alext\\Desktop\\School\\2021 Spring\\CS 271\\Final Project\\Feature Extraction\\Jazz\\Jazz Corpus'
export_path = 'C:\\Users\\alext\\Desktop\\School\\2021 Spring\\CS 271\\Final Project\\Feature Extraction\\Jazz\\Jazz Extracted Notes'

# Minimum number of notes per part
min_notes = 25

## Export a Music File to a CSV File

In [3]:
def export_file(corpus_path, export_path, file_name):
    # Construct the full path of the music file
    file_path = os.path.join(corpus_path, file_name);
    # Construct the full path of the target csv file
    target_path = os.path.join(export_path, file_name[:-4] + '.csv');
    
    # If the csv already exists, skip this music file
    if os.path.exists(target_path):
        return
    
    # Parse the file and convert it into a stream object
    score_stream = converter.parse(file_path)
    
    # The number of keyboard parts in the song
    keyboard_parts = 0
    
    # Attempt to separate the score by instrument
    try:
        # Iterate over all instrument parts in the score
        for part in instrument.partitionByInstrument(score_stream.flat):
            # Check for keyboard instruments
            print(part.getInstrument())
            if isinstance(part.getInstrument(), instrument.KeyboardInstrument):
                # Check if the part meets the minimum note requirement
                if len(part.elements) >= min_notes:
                    keyboard_parts += 1

                    # Remove the extension of the music file name
                    part_name = file_name[:-4]

                    # If this isn't the first keybaord part, append the part number to the part name
                    if keyboard_parts > 1:
                        part_name += '-' + str(keyboard_parts)

                    # Export the part to a csv
                    export_part(corpus_path, export_path, part, part_name)
    
    # If no instruments are specified, assume they are all piano
    except:
        export_part(corpus_path, export_path, score_stream.flat, file_name)

## Export a Part to a CSV File

In [4]:
def export_part(corpus_path, export_path, part, part_name):
    # Extract the notes of the part
    notes = extract_notes(part)
    
    # Construct the full path of the target csv file
    csv_path = os.path.join(export_path, part_name + '.csv');
    
    # Write the features list to a csv file
    with open(csv_path, 'w', newline='') as csv_file:
        write = csv.writer(csv_file)
        write.writerows(notes)

## Extract the Notes from a Part

In [5]:
def extract_notes(part):
    # Get an iterator of all notes and chords in the score
    note_stream = part.flat.notes
    
    # An orderd list of the notes in the song
    features = []
    
    # Keep looping through the stream until the end of the enumeration
    for phrase in takewhile(lambda x: True, note_stream):
        # A single note
        if type(phrase) is note.Note:
            features.append([phrase.pitch.midi])
        # A chord
        elif type(phrase) is chord.Chord:
            pitches = phrase.pitches
            midi_pitches = map(lambda x: x.midi, pitches)
            features.append(list(midi_pitches))
    
    return features

## Preprocess the Music Corpus

In [6]:
print('Extracting...\n')

# Go to the corpus directory
os.chdir(corpus_path)

# Get a list of all music files in the corpus
music_files = glob('*.mxl') + glob('*.mid')

num_errors = 0

# Export the notes of each score as a csv file
for file_name in music_files:
    try:
        export_file(corpus_path, export_path, file_name)
    except Exception as err:
        print("Skipping File {0}. Error: {1}".format(file_name, err))
        num_errors += 1

print('\nFinished')
print('Total number of errors: ' + str(num_errors))

Extracting...



Bass: Bass
Acoustic Bass
Piano: Piano
(<music21.instrument.Piano 'Piano: Piano'>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.key.Key of F major>, <music21.key.Key of F major>, <music21.key.Key of F major>, <music21.key.Key o